# Use CASH_OUT_Template Customized Notebook Template

In [12]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

import pandas as pd
import datetime as dt

In [13]:
#import seaborn as sns
import subprocess

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

In [14]:
from fosforml import register_model
import requests

In [15]:
# Import label encoder 
from sklearn import preprocessing 
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from math import sqrt
import numpy as np

In [16]:
table_name = 'ATM_TRANSACTION_MASTER_DATA'

sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [17]:
table_name = 'ATM_TRANSACTION_MASTER_DATA'

sf_df = my_session.sql("select * from {}".format(table_name))
df_original = sf_df.to_pandas()

In [18]:
#to_drop = ['ATM_STATE_DETAILS_STATE','MALE_POPULATION','FEMALE_POPULATION','TOTAL_POPULATION','NO_OF_VETERANS','FOREIGN_BORN','AVG_HOUSEHOLD_SIZE','LANDAREA_SQ_MILE','POPULATION_RATIO','POPULATION_DENSITY','MEDIAN_HOUSE_INCOME']
to_drop = ['MALE_POPULATION','FEMALE_POPULATION','TOTAL_POPULATION','NO_OF_VETERANS','FOREIGN_BORN','AVG_HOUSEHOLD_SIZE','LANDAREA_SQ_MILE','POPULATION_RATIO','POPULATION_DENSITY','MEDIAN_HOUSE_INCOME']

df.drop(to_drop, inplace=True, axis=1)

In [19]:
#to_drop = ['ATM_STATE_DETAILS_STATE','MALE_POPULATION','FEMALE_POPULATION','TOTAL_POPULATION','NO_OF_VETERANS','FOREIGN_BORN','AVG_HOUSEHOLD_SIZE','LANDAREA_SQ_MILE','POPULATION_RATIO','POPULATION_DENSITY','MEDIAN_HOUSE_INCOME']
to_drop = ['MALE_POPULATION','FEMALE_POPULATION','TOTAL_POPULATION','NO_OF_VETERANS','FOREIGN_BORN','AVG_HOUSEHOLD_SIZE','LANDAREA_SQ_MILE','POPULATION_RATIO','POPULATION_DENSITY','MEDIAN_HOUSE_INCOME']
df_original.drop(to_drop, inplace=True, axis=1)

In [20]:
df.shape

(312417, 16)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312417 entries, 0 to 312416
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   DATE                     312417 non-null  object 
 1   WEEKDAY_FLAG             312417 non-null  object 
 2   HOLIDAY_FLAG             312417 non-null  object 
 3   BANK_ID                  312417 non-null  int8   
 4   STATE                    312417 non-null  object 
 5   BANK_NAME                312417 non-null  object 
 6   ATM_ID                   312417 non-null  object 
 7   DISPENSED_AMOUNT         312417 non-null  float64
 8   TOTAL_TRANSACTION_COUNT  312417 non-null  float64
 9   DOWNTIME_IN_MINS         312417 non-null  float64
 10  ATM_MAX_CAPACITY         312417 non-null  float64
 11  LOCATION_TYPE            312417 non-null  object 
 12  ATM_PLACEMENTS           312417 non-null  object 
 13  ATM_ACCESSIBILITY        312417 non-null  object 
 14  ATM_

In [22]:
df.head()

,DATE,WEEKDAY_FLAG,HOLIDAY_FLAG,BANK_ID,STATE,BANK_NAME,ATM_ID,DISPENSED_AMOUNT,TOTAL_TRANSACTION_COUNT,DOWNTIME_IN_MINS,ATM_MAX_CAPACITY,LOCATION_TYPE,ATM_PLACEMENTS,ATM_ACCESSIBILITY,ATM_MAINTAINENANCE,ATM__REPLENISHMENT
0,2023-01-29,Y,N,3,Maine,Citibank,TBH000274025,225180.0,90.0,0.0,2860000.0,Urban,Drive-Thru,Restrcited hours,Irregular,Weekly
1,2023-01-30,Y,N,3,Maine,Citibank,TBH000274025,262440.0,117.0,0.0,2860000.0,Urban,Drive-Thru,Restrcited hours,Irregular,Weekly
2,2023-01-31,Y,N,3,Maine,Citibank,TBH000274025,561150.0,160.0,0.0,2860000.0,Urban,Drive-Thru,Restrcited hours,Irregular,Weekly
3,2023-02-01,Y,N,3,Maine,Citibank,TBH000274025,437220.0,129.0,0.0,2860000.0,Urban,Drive-Thru,Restrcited hours,Irregular,Weekly
4,2023-02-02,Y,N,3,Maine,Citibank,TBH000274025,360900.0,119.0,0.0,2860000.0,Urban,Drive-Thru,Restrcited hours,Irregular,Weekly


In [23]:
df.columns

Index(['DATE', 'WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'BANK_NAME',
       'ATM_ID', 'DISPENSED_AMOUNT', 'TOTAL_TRANSACTION_COUNT',
       'DOWNTIME_IN_MINS', 'ATM_MAX_CAPACITY', 'LOCATION_TYPE',
       'ATM_PLACEMENTS', 'ATM_ACCESSIBILITY', 'ATM_MAINTAINENANCE',
       'ATM__REPLENISHMENT'],
      dtype='object')

In [24]:
df.isnull().sum()

DATE                       0
WEEKDAY_FLAG               0
HOLIDAY_FLAG               0
BANK_ID                    0
STATE                      0
BANK_NAME                  0
ATM_ID                     0
DISPENSED_AMOUNT           0
TOTAL_TRANSACTION_COUNT    0
DOWNTIME_IN_MINS           0
ATM_MAX_CAPACITY           0
LOCATION_TYPE              0
ATM_PLACEMENTS             0
ATM_ACCESSIBILITY          0
ATM_MAINTAINENANCE         0
ATM__REPLENISHMENT         0
dtype: int64

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312417 entries, 0 to 312416
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   DATE                     312417 non-null  object 
 1   WEEKDAY_FLAG             312417 non-null  object 
 2   HOLIDAY_FLAG             312417 non-null  object 
 3   BANK_ID                  312417 non-null  int8   
 4   STATE                    312417 non-null  object 
 5   BANK_NAME                312417 non-null  object 
 6   ATM_ID                   312417 non-null  object 
 7   DISPENSED_AMOUNT         312417 non-null  float64
 8   TOTAL_TRANSACTION_COUNT  312417 non-null  float64
 9   DOWNTIME_IN_MINS         312417 non-null  float64
 10  ATM_MAX_CAPACITY         312417 non-null  float64
 11  LOCATION_TYPE            312417 non-null  object 
 12  ATM_PLACEMENTS           312417 non-null  object 
 13  ATM_ACCESSIBILITY        312417 non-null  object 
 14  ATM_

In [26]:
df['DATE'] = pd.to_datetime(df['DATE'])
df['HOUR'] = df['DATE'].dt.hour
df['DAYOFWEEK'] = df['DATE'].dt.dayofweek
df['QUARTER'] = df['DATE'].dt.quarter
df['MONTH'] = df['DATE'].dt.month
df['YEAR'] = df['DATE'].dt.year
df['DAYOFYEAR'] = df['DATE'].dt.dayofyear
df['DAYOFMONTH'] = df['DATE'].dt.day
df['DATE'] = pd.to_datetime(df['DATE']).dt.strftime("%Y-%m-%d %H:%M:%S.%f")

In [27]:
df['WEEKDAY_FLAG'] = df['WEEKDAY_FLAG'].map({'Y': 1, 'N': 0})
df['HOLIDAY_FLAG'] = df['HOLIDAY_FLAG'].map({'Y': 1, 'N': 0})

In [28]:
df.head()

,DATE,WEEKDAY_FLAG,HOLIDAY_FLAG,BANK_ID,STATE,BANK_NAME,ATM_ID,DISPENSED_AMOUNT,TOTAL_TRANSACTION_COUNT,DOWNTIME_IN_MINS,...,ATM_ACCESSIBILITY,ATM_MAINTAINENANCE,ATM__REPLENISHMENT,HOUR,DAYOFWEEK,QUARTER,MONTH,YEAR,DAYOFYEAR,DAYOFMONTH
0,2023-01-29 00:00:00.000000,1,0,3,Maine,Citibank,TBH000274025,225180.0,90.0,0.0,...,Restrcited hours,Irregular,Weekly,0,6,1,1,2023,29,29
1,2023-01-30 00:00:00.000000,1,0,3,Maine,Citibank,TBH000274025,262440.0,117.0,0.0,...,Restrcited hours,Irregular,Weekly,0,0,1,1,2023,30,30
2,2023-01-31 00:00:00.000000,1,0,3,Maine,Citibank,TBH000274025,561150.0,160.0,0.0,...,Restrcited hours,Irregular,Weekly,0,1,1,1,2023,31,31
3,2023-02-01 00:00:00.000000,1,0,3,Maine,Citibank,TBH000274025,437220.0,129.0,0.0,...,Restrcited hours,Irregular,Weekly,0,2,1,2,2023,32,1
4,2023-02-02 00:00:00.000000,1,0,3,Maine,Citibank,TBH000274025,360900.0,119.0,0.0,...,Restrcited hours,Irregular,Weekly,0,3,1,2,2023,33,2


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312417 entries, 0 to 312416
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   DATE                     312417 non-null  object 
 1   WEEKDAY_FLAG             312417 non-null  int64  
 2   HOLIDAY_FLAG             312417 non-null  int64  
 3   BANK_ID                  312417 non-null  int8   
 4   STATE                    312417 non-null  object 
 5   BANK_NAME                312417 non-null  object 
 6   ATM_ID                   312417 non-null  object 
 7   DISPENSED_AMOUNT         312417 non-null  float64
 8   TOTAL_TRANSACTION_COUNT  312417 non-null  float64
 9   DOWNTIME_IN_MINS         312417 non-null  float64
 10  ATM_MAX_CAPACITY         312417 non-null  float64
 11  LOCATION_TYPE            312417 non-null  object 
 12  ATM_PLACEMENTS           312417 non-null  object 
 13  ATM_ACCESSIBILITY        312417 non-null  object 
 14  ATM_

In [30]:
# how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['STATE']= label_encoder.fit_transform(df['STATE'])

In [31]:
# how to understand word labels. 
atm_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['ATM_ID']= atm_encoder.fit_transform(df['ATM_ID'])

In [32]:
df.drop(['DATE','BANK_NAME','TOTAL_TRANSACTION_COUNT','DOWNTIME_IN_MINS','ATM_MAX_CAPACITY'], axis=1,inplace=True)

In [33]:
df.columns

Index(['WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'ATM_ID',
       'DISPENSED_AMOUNT', 'LOCATION_TYPE', 'ATM_PLACEMENTS',
       'ATM_ACCESSIBILITY', 'ATM_MAINTAINENANCE', 'ATM__REPLENISHMENT', 'HOUR',
       'DAYOFWEEK', 'QUARTER', 'MONTH', 'YEAR', 'DAYOFYEAR', 'DAYOFMONTH'],
      dtype='object')

In [34]:
features = ['WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'ATM_ID',
    'HOUR', 'DAYOFWEEK', 'QUARTER', 'MONTH', 'YEAR',
       'DAYOFYEAR', 'DAYOFMONTH']

In [35]:
print (df.shape)
df = df[df['DISPENSED_AMOUNT'] > 0]
print (df.shape)

(312417, 18)
(257564, 18)


In [36]:
X = df[features]
y = df['DISPENSED_AMOUNT']

In [37]:
X.head()

,WEEKDAY_FLAG,HOLIDAY_FLAG,BANK_ID,STATE,ATM_ID,HOUR,DAYOFWEEK,QUARTER,MONTH,YEAR,DAYOFYEAR,DAYOFMONTH
0,1,0,3,18,299,0,6,1,1,2023,29,29
1,1,0,3,18,299,0,0,1,1,2023,30,30
2,1,0,3,18,299,0,1,1,1,2023,31,31
3,1,0,3,18,299,0,2,1,2,2023,32,1
4,1,0,3,18,299,0,3,1,2,2023,33,2


In [38]:
y

0         225180.0
1         262440.0
2         561150.0
3         437220.0
4         360900.0
            ...   
312412    195300.0
312413    245160.0
312414    222480.0
312415    198360.0
312416    314190.0
Name: DISPENSED_AMOUNT, Length: 257564, dtype: float64

In [39]:
# Split the data into training and test sets. (0.75, 0.25) split.
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)

In [40]:
print(f'Total # of sample in whole dataset: {len(X)}')
print("*****"*10)
print(f'Total # of sample in train dataset: {len(X_train)}')
print(f'Shape of X_train: {X_train.shape}')
print("*****"*10)
print(f'Total # of sample in test dataset: {len(X_test)}')
print(f'Shape of X_test: {X_test.shape}')

Total # of sample in whole dataset: 257564
**************************************************
Total # of sample in train dataset: 193173
Shape of X_train: (193173, 12)
**************************************************
Total # of sample in test dataset: 64391
Shape of X_test: (64391, 12)


# Random Forest

In [41]:
tree = RandomForestRegressor(max_depth=4,max_features=4)

In [42]:
tree.fit(X_train, y_train)

RandomForestRegressor(max_depth=4, max_features=4)

In [43]:
predictions = tree.predict(X_test)

In [44]:
print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

Mean Absolute Error: 178584.92386534007
Mean Squared Error: 52230399752.92698
Root Mean Squared Error: 228539.71154468314
r2_score: 0.24791762344601276


In [45]:
tree.feature_importances_

array([9.75678177e-04, 2.07487548e-05, 6.98055928e-02, 2.55204678e-02,
       6.44508435e-01, 0.00000000e+00, 7.17736493e-03, 1.00469971e-02,
       1.01290357e-02, 1.90417477e-02, 1.85112109e-02, 1.94262721e-01])

In [46]:
pd.Series(tree.feature_importances_,index=features).sort_values(ascending=False)

ATM_ID          0.644508
DAYOFMONTH      0.194263
BANK_ID         0.069806
STATE           0.025520
YEAR            0.019042
DAYOFYEAR       0.018511
MONTH           0.010129
QUARTER         0.010047
DAYOFWEEK       0.007177
WEEKDAY_FLAG    0.000976
HOLIDAY_FLAG    0.000021
HOUR            0.000000
dtype: float64

In [47]:
param_grid = [{"max_depth":[3,4,5, None], "max_features":[3,4,5,6,7]}]

In [48]:
gs = GridSearchCV(estimator=RandomForestRegressor(random_state=123),param_grid = param_grid,cv=10)

In [ ]:
gs.fit(X_train, y_train)

In [ ]:
gs.cv_results_['params']

In [ ]:
gs.cv_results_['rank_test_score']

In [ ]:
gs.best_estimator_

In [ ]:
#Build Tree using Best parameters

tree1 = RandomForestRegressor(max_features=7, random_state=123)
tree1.fit(X_train, y_train)
predictions = tree1.predict(X_test)
y_pred = predictions

In [ ]:
X_train.columns

In [ ]:
print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

In [ ]:
y_train_df = pd.DataFrame(y_train)
y_test_df = pd.DataFrame(y_test)
y_pred_df = pd.DataFrame(y_pred, columns=['PREDICTION'])

In [51]:
type(X_train), type(X_test), type(y_train_df), type(y_test_df), type(y_pred_df)

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.series.Series,
 pandas.core.series.Series,
 numpy.ndarray,
 pandas.core.series.Series)

In [ ]:
atm_train_sf=my_session.createDataFrame(
        X_tra.values.tolist(),
        schema=insurance.columns.tolist())
atm_train_sf.write.mode("overwrite").save_as_table("FDC_Insurance.INS_CUST_SEGMT_SCHEMA.INSURANCE_DETAILS")

In [50]:
## registering the model in Fosfor Insight Designer.
register_model(
    model_obj=tree1, 
    session=my_session,
    x_train=X_train,
    y_train=y_train_df,
    x_test=X_test,
    y_test=y_test_df,
    y_pred=y_pred_df,
    source="Notebook",
    dataset_name="ATM_TRANSACTION_MASTER_DATA",
    dataset_source="Snowflake",
    #dataset_source="InMemory",
    name="RandForest_Dispense_Amount_Regressor",
    description="ATM_DispenseAMT_RandomForest_Regression",
    flavour="sklearn",
    model_type="regression",
    conda_dependencies=["scikit-learn==1.3.2"]
)

NameError: name 'tree1' is not defined